<a href="https://colab.research.google.com/github/dripston/fake-news-ml-project/blob/main/fakenewsprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#importing kaggle.json ZipFile
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rehaann","key":"76d0365f78bfc54051548159ce83f0c8"}'}

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [9]:
!kaggle competitions download -c fakenewskdd2020

  0% 0.00/9.16M [00:00<?, ?B/s]
100% 9.16M/9.16M [00:00<00:00, 855MB/s]


In [10]:
from zipfile import ZipFile

with ZipFile('/content/fakenewskdd2020.zip', 'r') as z:
    z.extractall('/content/')

In [11]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

DATA PRE-PROCESSING

In [14]:
# quick peek at the raw file to inspect problematic formatting
with open('/content/train.csv', 'r', encoding='utf-8', errors='replace') as f:
    for i, line in enumerate(f):
        print(i+1, repr(line[:500]))   # show first 500 chars of each line
        if i >= 19:
            break


1 'text\tlabel\n'
2 'Get the latest from TODAY Sign up for our newsletter  No one ever truly gets over losing a loved one, and Blake Shelton is no exception. He was just 14 when his older brother Richie died on Nov. 13, 1990. And, as Shelton noted in a tweet Monday, "It changed my life forever."  Richie was 24 when he died in a car accident in the Sheltons\' home state of Oklahoma. Two years ago, Shelton sent out a message for the 25th anniversary of his loss:  Richie, who was Blake\'s half-brother (they shared a moth'
3 '2d  Conan On The Funeral Trump Will Be Invited To - CONAN on TBS\t1\n'
4 'It’s safe to say that Instagram Stories has far surpassed its competitor Snapchat in popularity since it’s inception two years ago—and your favorite celebrities have hopped on the social media trend. Unlike a highly curated photo feed, Instagram Stories is where celebrities seem to be comfortable enough to be raw and open.  Need something to do while you’re waiting in line or on a short break? T

In [15]:
#loading the dataset to a pandas dataframe
news_dataset=pd.read_csv('/content/train.csv',sep='\t',on_bad_lines="skip", engine='python', encoding='utf-8')


In [16]:
news_dataset.head()

,text,label
0,Get the latest from TODAY Sign up for our news...,1
1,2d Conan On The Funeral Trump Will Be Invited...,1
2,It’s safe to say that Instagram Stories has fa...,0
3,Much like a certain Amazon goddess with a lass...,0
4,At a time when the perfect outfit is just one ...,0


In [17]:
news_dataset.shape

(4923, 2)

In [18]:
#counting the number of missing values in the datasset
news_dataset.isnull().sum()

,0
text,0
label,0


In [19]:
#seperating the data and label
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset["label"]
print(X)
print(Y)

                                                   text
0     Get the latest from TODAY Sign up for our news...
1     2d  Conan On The Funeral Trump Will Be Invited...
2     It’s safe to say that Instagram Stories has fa...
3     Much like a certain Amazon goddess with a lass...
4     At a time when the perfect outfit is just one ...
...                                                 ...
4918  The storybook romance of WWE stars John Cena a...
4919  The actor told friends he’s responsible for en...
4920  Sarah Hyland is getting real.  The Modern Fami...
4921  Production has been suspended on the sixth and...
4922  A jury ruled against Bill Cosby in his sexual ...

[4923 rows x 1 columns]
0       1
1       1
2       0
3       0
4       0
       ..
4918    0
4919    0
4920    0
4921    0
4922    0
Name: label, Length: 4923, dtype: object


Stemming:
stemming is a process of reducing a word to its root word

In [20]:
port_stem=PorterStemmer()

In [21]:
def stemming(text):
  stemmed_text=re.sub('[^a-zA-Z]',' ',text)
  stemmed_text=stemmed_text.lower()
  stemmed_text=stemmed_text.split()
  stemmed_text=[port_stem.stem(word) for word in stemmed_text if not word in stopwords.words('english')]
  stemmed_text=' '.join(stemmed_text)
  return stemmed_text


In [22]:
news_dataset['text']=news_dataset['text'].apply(stemming)

In [23]:
print(news_dataset['text'])

0       get latest today sign newslett one ever truli ...
1                        conan funer trump invit conan tb
2       safe say instagram stori far surpass competito...
3       much like certain amazon goddess lasso height ...
4       time perfect outfit one click away high demand...
                              ...                        
4918    storybook romanc wwe star john cena nikki bell...
4919    actor told friend respons encourag brad reigni...
4920    sarah hyland get real modern famili star took ...
4921    product suspend sixth final season netflix hou...
4922    juri rule bill cosbi sexual assault retrial th...
Name: text, Length: 4923, dtype: object


In [24]:
#seperating the data and label
X=news_dataset['text'].values
Y=news_dataset['label'].values

In [25]:
print(X)

['get latest today sign newslett one ever truli get lose love one blake shelton except older brother richi die nov shelton note tweet monday chang life forev richi die car accid shelton home state oklahoma two year ago shelton sent messag th anniversari loss richi blake half brother share mother passeng car collid school bu ada south oklahoma citi richi driver redena mcmanu year old boy christoph mcmanu die shortli collis bu driver passeng uninjur accord polic report accid clearli remain blake told minut rememb pick phone call week dead tell someth pick phone call tell someth saw tv like constantli shock dead blake shelton play today halloween extravaganza new york citi oct getti imag blake wife miranda lambert wrote singl call inspir richi still two brother bond despit age differ share love countri music bedroom right across hallway mine littl blake said interview listen hank william jr waylon lynyrd skynyrd bob seeger whatev popular realli richi love music would sit go man guy hero c

In [26]:
print(Y)

['1' '1' '0' ... '0' '0' '0']


In [27]:
#converting the textual data into numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)

In [28]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 941099 stored elements and shape (4923, 42539)>
  Coords	Values
  (0, 171)	0.12494765923889722
  (0, 183)	0.03127269919847014
  (0, 253)	0.04847583923509015
  (0, 276)	0.0920822342378783
  (0, 520)	0.042201484082939805
  (0, 552)	0.041878966418217804
  (0, 1402)	0.0569020028697862
  (0, 3086)	0.06573138244138836
  (0, 3587)	0.03445479698534219
  (0, 3817)	0.35691460806983283
  (0, 4024)	0.06165980053862364
  (0, 4144)	0.05729842705359366
  (0, 4376)	0.04406216415727539
  (0, 4823)	0.1737576506790321
  (0, 4904)	0.14062122162146964
  (0, 5381)	0.08637318103765103
  (0, 5612)	0.09811887783328178
  (0, 6243)	0.036795968744863065
  (0, 6727)	0.06177300702669815
  (0, 6872)	0.08032076339585868
  (0, 6995)	0.051803457419479816
  (0, 7316)	0.0864396150648863
  (0, 7319)	0.08806198729525
  (0, 7682)	0.06154757420560213
  (0, 7794)	0.0864396150648863
  :	:
  (4922, 37779)	0.013380265914774718
  (4922, 38042)	0.01123489291027521
  (49

In [33]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

In [34]:
print(X_train)
print(Y_train)
print(Y_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 757555 stored elements and shape (3938, 42539)>
  Coords	Values
  (0, 183)	0.020798354207228024
  (0, 275)	0.020914165897311515
  (0, 651)	0.043923327926448825
  (0, 796)	0.044354447909435483
  (0, 983)	0.014599451884613774
  (0, 1308)	0.02595795425963519
  (0, 1405)	0.022108074440748218
  (0, 2123)	0.034846985799174605
  (0, 2407)	0.0254870441139362
  (0, 2449)	0.017465087579507498
  (0, 2490)	0.057024790687121664
  (0, 2591)	0.06646585810442897
  (0, 3131)	0.04302557609230499
  (0, 3425)	0.02037056673951725
  (0, 3587)	0.02291465368215915
  (0, 3763)	0.15305943300112326
  (0, 3847)	0.0830982211584007
  (0, 3936)	0.18370467869527152
  (0, 4257)	0.029151250752632852
  (0, 4376)	0.029304170115410817
  (0, 4708)	0.02830382219306619
  (0, 4746)	0.13208731057136158
  (0, 5153)	0.10877433029711042
  (0, 5381)	0.019147904927396202
  (0, 5608)	0.03988476440695158
  :	:
  (3937, 33948)	0.07879217078164776
  (3937, 34179)	0.048653092

In [35]:
model=LogisticRegression()

In [36]:
#training the model
model.fit(X_train,Y_train)

LogisticRegression()

In [37]:
#Evaluation of the model at accuracy score
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [39]:
print(X_train_prediction)
print(training_data_accuracy)

['0' '0' '1' ... '0' '1' '0']
0.8577958354494667


In [40]:

X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)
print(test_data_accuracy)

0.7979695431472081


In [58]:
X_new = X_test[503].reshape(1, -1)
prediction = model.predict(X_new)
print(prediction)

if int(prediction[0]) == 0:
    print('The news is Real')
else:
    print('The news is Fake')

['1']
The news is Fake


In [57]:
print(Y_test[503])

1


In [ ]:
# This cell was intentionally left empty and was not part of the original notebook.
# It is being used by the AI for output generation.

## Detailed Code Explanation

Here's a step-by-step breakdown of the code in this notebook:

1.  **Importing Libraries and Downloading Data:** This section imports necessary libraries and downloads the dataset from Kaggle. It also includes initial data inspection.